In [5]:
sys.executable

'/Users/utkarshtripathi/opt/anaconda3/bin/python'

In [6]:
!pip install alpaca-py


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.8/97.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 26.6 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import sys
!{sys.executable} -m pip install alpaca


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [28]:
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
import csv
import requests
import pandas as pd
import io
import ssl
import websocket
import _thread

In [29]:
PAPER_ENDPOINT = "https://paper-api.alpaca.markets"
API_KEY = "PK8ZOSEE0UA91VMR6JBB"
SECRET_KEY = "GC79IfVdqr7UBt16dkvYcsQh6BAIcahGnwwnUtfR"
HEADERS = {
    'APCA-API-KEY-ID': API_KEY,
    'APCA-API-SECRET-KEY': SECRET_KEY
}

ALPHA_API_KEY = "OG8JDKQOTOISF1RD"

In [30]:
HEADERS

{'APCA-API-KEY-ID': 'PK8ZOSEE0UA91VMR6JBB',
 'APCA-API-SECRET-KEY': 'GC79IfVdqr7UBt16dkvYcsQh6BAIcahGnwwnUtfR'}

In [31]:
trading_client = TradingClient(API_KEY, SECRET_KEY, paper=True)

In [32]:
account = trading_client.get_account()
for property_name, value in account:
    print(f"\"{property_name}\": {value}")

"id": 27276f02-97a3-4ce6-8506-b2ba3c086454
"account_number": PA3G5KATTY1Q
"status": ACTIVE
"crypto_status": ACTIVE
"currency": USD
"buying_power": 199489.87
"regt_buying_power": 199489.87
"daytrading_buying_power": 0
"non_marginable_buying_power": 99510.98
"cash": 99510.98
"accrued_fees": 0
"pending_transfer_out": None
"pending_transfer_in": 0
"portfolio_value": 99978.89
"pattern_day_trader": False
"trading_blocked": False
"transfers_blocked": False
"account_blocked": False
"created_at": 2022-12-02 19:19:52.938885+00:00
"trade_suspended_by_user": False
"multiplier": 2
"shorting_enabled": True
"equity": 99978.89
"last_equity": 99986.69
"long_market_value": 467.91
"short_market_value": 0
"initial_margin": 233.955
"maintenance_margin": 140.373
"last_maintenance_margin": 160.96
"sma": 99927.52
"daytrade_count": 0


In [67]:
ticker = 'AMD'

In [70]:
market_order_data = MarketOrderRequest(
                      symbol=f"{ticker}",
                      notional=1000,
                      side=OrderSide.SELL,
                      time_in_force=TimeInForce.DAY
                  )

In [71]:
market_order = trading_client.submit_order(market_order_data)
for property_name, value in market_order:
    print(f"\"{property_name}\": {value}")

APIError: {"code":42210000,"message":"fractional orders cannot be sold short"}

In [34]:
# Get all open positions and print each of them
positions = trading_client.get_all_positions()
for position in positions:
    for property_name, value in position:
        print(f"\"{property_name}\": {value}")
    print("next")

"asset_id": b0b6dd9d-8b9b-48a9-ba46-b9d54906e415
"symbol": AAPL
"exchange": NASDAQ
"asset_class": us_equity
"avg_entry_price": 147.49
"qty": 2
"side": long
"market_value": 288.96
"cost_basis": 294.98
"unrealized_pl": -6.02
"unrealized_plpc": -0.0204081632653061
"unrealized_intraday_pl": -4.3
"unrealized_intraday_plpc": -0.0146627565982405
"current_price": 144.48
"lastday_price": 146.63
"change_today": -0.0146627565982405
next
"asset_id": 8ccae427-5dd0-45b3-b5fe-7ba5e422c766
"symbol": TSLA
"exchange": NASDAQ
"asset_class": us_equity
"avg_entry_price": 194.04
"qty": 1
"side": long
"market_value": 179.13
"cost_basis": 194.04
"unrealized_pl": -14.91
"unrealized_plpc": -0.0768398268398268
"unrealized_intraday_pl": -3.32
"unrealized_intraday_plpc": -0.0181967662373253
"current_price": 179.13
"lastday_price": 182.45
"change_today": -0.0181967662373253
next


In [57]:
symbol = 'AAPL'

In [48]:
apiserver_domain = 'paper-api.alpaca.markets'
url =  f'https://{apiserver_domain}/v2/positions/{symbol}'

In [49]:
r = requests.get(url,headers=HEADERS)

In [50]:
print(r.content)

b'{"asset_id":"b0b6dd9d-8b9b-48a9-ba46-b9d54906e415","symbol":"AAPL","exchange":"NASDAQ","asset_class":"us_equity","asset_marginable":false,"qty":"2","avg_entry_price":"147.49","side":"long","market_value":"289.16","cost_basis":"294.98","unrealized_pl":"-5.82","unrealized_plpc":"-0.0197301511966913","unrealized_intraday_pl":"-4.1","unrealized_intraday_plpc":"-0.0139807679192525","current_price":"144.58","lastday_price":"146.63","change_today":"-0.0139807679192525","qty_available":"2"}'


In [51]:
url =  f'https://{apiserver_domain}/v2/positions/{symbol}'
x = requests.delete(url,headers=HEADERS)
print(x.text)

{"id":"7d7b68a9-7184-453e-8230-771372e821f7","client_order_id":"7ec4918e-7c25-495d-b4bf-8c17d71711b8","created_at":"2022-12-06T16:32:19.073041689Z","updated_at":"2022-12-06T16:32:19.073089729Z","submitted_at":"2022-12-06T16:32:19.072527499Z","filled_at":null,"expired_at":null,"canceled_at":null,"failed_at":null,"replaced_at":null,"replaced_by":null,"replaces":null,"asset_id":"b0b6dd9d-8b9b-48a9-ba46-b9d54906e415","symbol":"AAPL","asset_class":"us_equity","notional":null,"qty":"2","filled_qty":"0","filled_avg_price":null,"order_class":"","order_type":"market","type":"market","side":"sell","time_in_force":"day","limit_price":null,"stop_price":null,"status":"pending_new","extended_hours":false,"legs":null,"trail_percent":null,"trail_price":null,"hwm":null,"subtag":null,"source":null}


In [52]:
print(x.content)

b'{"id":"7d7b68a9-7184-453e-8230-771372e821f7","client_order_id":"7ec4918e-7c25-495d-b4bf-8c17d71711b8","created_at":"2022-12-06T16:32:19.073041689Z","updated_at":"2022-12-06T16:32:19.073089729Z","submitted_at":"2022-12-06T16:32:19.072527499Z","filled_at":null,"expired_at":null,"canceled_at":null,"failed_at":null,"replaced_at":null,"replaced_by":null,"replaces":null,"asset_id":"b0b6dd9d-8b9b-48a9-ba46-b9d54906e415","symbol":"AAPL","asset_class":"us_equity","notional":null,"qty":"2","filled_qty":"0","filled_avg_price":null,"order_class":"","order_type":"market","type":"market","side":"sell","time_in_force":"day","limit_price":null,"stop_price":null,"status":"pending_new","extended_hours":false,"legs":null,"trail_percent":null,"trail_price":null,"hwm":null,"subtag":null,"source":null}'


In [63]:
params = {"symbol":symbol, "notional": "1000", "side":"buy", "type":"market","time_in_force":"day"}

In [64]:
url =  f'https://{apiserver_domain}/v2/orders'
o = requests.post(url, json=params,headers=HEADERS)

In [65]:
print(o.text)

{"id":"5b9becea-3d33-482c-b41d-3a307673a244","client_order_id":"2acde4e6-4f30-4545-9ce2-ecae48c17746","created_at":"2022-12-06T17:07:04.014437115Z","updated_at":"2022-12-06T17:07:04.014475285Z","submitted_at":"2022-12-06T17:07:04.013480135Z","filled_at":null,"expired_at":null,"canceled_at":null,"failed_at":null,"replaced_at":null,"replaced_by":null,"replaces":null,"asset_id":"b0b6dd9d-8b9b-48a9-ba46-b9d54906e415","symbol":"AAPL","asset_class":"us_equity","notional":"1000","qty":null,"filled_qty":"0","filled_avg_price":null,"order_class":"","order_type":"market","type":"market","side":"buy","time_in_force":"day","limit_price":null,"stop_price":null,"status":"pending_new","extended_hours":false,"legs":null,"trail_percent":null,"trail_price":null,"hwm":null,"subtag":null,"source":null}


In [33]:
data.keys()

dict_keys(['Meta Data', 'Time Series (5min)'])

In [31]:
data['Meta Data']

{'1. Information': 'Intraday (5min) open, high, low, close prices and volume',
 '2. Symbol': 'AAPL',
 '3. Last Refreshed': '2022-12-02 20:00:00',
 '4. Interval': '5min',
 '5. Output Size': 'Full size',
 '6. Time Zone': 'US/Eastern'}

In [ ]:
data['Meta Data']

In [36]:
pd.DataFrame.from_dict(data['Time Series (5min)'], orient='index')

,1. open,2. high,3. low,4. close,5. volume
2022-12-02 20:00:00,147.5100,147.5100,147.4200,147.5000,4270
2022-12-02 19:55:00,147.5000,147.5000,147.5000,147.5000,549
2022-12-02 19:50:00,147.4500,147.5000,147.4300,147.5000,4718
2022-12-02 19:45:00,147.4900,147.4900,147.4500,147.4500,903
2022-12-02 19:35:00,147.4500,147.4600,147.4200,147.4600,1628
...,...,...,...,...,...
2022-11-04 04:25:00,138.5500,138.6900,138.5500,138.6300,4095
2022-11-04 04:20:00,138.7700,138.8800,138.6900,138.6900,9184
2022-11-04 04:15:00,138.6300,138.8000,138.5100,138.7600,13289
2022-11-04 04:10:00,138.6600,138.7000,138.5100,138.5100,13920


In [19]:
ws = websocket.WebSocket(sslopt={"cert_reqs": ssl.CERT_NONE})
ws.connect("wss://stream.data.alpaca.markets/v2/iex")

In [20]:
print(ws.recv())

[{"T":"success","msg":"connected"}]


In [21]:
authenticate_str = '{"action": "auth", "key": "PK8ZOSEE0UA91VMR6JBB", "secret": "GC79IfVdqr7UBt16dkvYcsQh6BAIcahGnwwnUtfR"}'

In [22]:
authenticate_str

'{"action": "auth", "key": "PK8ZOSEE0UA91VMR6JBB", "secret": "GC79IfVdqr7UBt16dkvYcsQh6BAIcahGnwwnUtfR"}'

In [23]:
ws.send(authenticate_str)

109

In [24]:
print(ws.recv())

[{"T":"success","msg":"authenticated"}]


In [25]:
subscription_str = f'{{"action":"subscribe","trades":["AAPL"],"quotes":["AMD","CLDR"],"bars":["AAPL","VOO"]}}'

In [26]:
subscription_str

'{"action":"subscribe","trades":["AAPL"],"quotes":["AMD","CLDR"],"bars":["AAPL","VOO"]}'

In [27]:
ws.send(subscription_str)

92

In [28]:
print(ws.recv())

[{"T":"subscription","trades":["AAPL"],"quotes":["AMD","CLDR"],"bars":["AAPL","VOO"],"updatedBars":[],"dailyBars":[],"statuses":[],"lulds":[],"corrections":["AAPL"],"cancelErrors":["AAPL"]}]


In [29]:
def on_message(ws, message):
    print(message)

def on_error(ws, error):
    print(error)

def on_close(ws, close_status_code, close_msg):
    print("### closed ###")

def on_open(ws):
    print("Opened connection")

if __name__ == "__main__":
    websocket.enableTrace(True)
    ws = websocket.WebSocketApp("wss://stream.data.alpaca.markets/v2/iex",
                              on_open=on_open,
                              on_message=on_message,
                              on_error=on_error,
                              on_close=on_close)

    ws.run_forever()  # Set dispatcher to automatic reconnection, 5 second reconnect delay if connection closed unexpectedly

--- request header ---
GET /v2/iex HTTP/1.1
Upgrade: websocket
Host: stream.data.alpaca.markets
Origin: https://stream.data.alpaca.markets
Sec-WebSocket-Key: Sjvy5IydbYYgxSGH45VWNA==
Sec-WebSocket-Version: 13
Connection: Upgrade


-----------------------
--- response header ---
HTTP/1.1 101 Switching Protocols
Date: Sat, 03 Dec 2022 22:47:26 GMT
Connection: upgrade
Upgrade: websocket
Sec-WebSocket-Accept: MWViEAca3lzXZLB/DQIGVmoTXYQ=
Strict-Transport-Security: max-age=15724800; includeSubDomains
-----------------------
++Rcv raw: b'\x81#[{"T":"success","msg":"connected"}]'
++Rcv decoded: fin=1 opcode=1 data=b'[{"T":"success","msg":"connected"}]'


Opened connection
[{"T":"success","msg":"connected"}]


++Rcv raw: b'\x81/[{"T":"error","code":404,"msg":"auth timeout"}]'
++Rcv decoded: fin=1 opcode=1 data=b'[{"T":"error","code":404,"msg":"auth timeout"}]'


[{"T":"error","code":404,"msg":"auth timeout"}]
Connection to remote host was lost.
### closed ###


In [74]:
import websocket, json

def on_open(ws):
    print("opened")
    auth_data = {
        "action": "authenticate",
        "data": {"key_id": API_KEY, "secret_key": SECRET_KEY}
    }

    ws.send(json.dumps(auth_data))

    # channel_data = {
    #     "action": "subscribe",
    #     "params": TICKERS
    # }

    # ws.send(json.dumps(channel_data))


def on_message(ws, message):
    print("received a message")
    print(message)

def on_close(ws):
    print("closed connection")

socket = "wss://data.alpaca.markets/stream"

ws = websocket.WebSocketApp(socket, on_open=on_open)
# ws = websocket.WebSocketApp(socket, on_open=on_open, on_message=on_message, on_close=on_close)
ws.run_forever()

True